In [2]:
!pip -qqq install yt-dlp

In [3]:
from yt_dlp import YoutubeDL
import os
import re

In [ ]:
#!pip install moviepy
#!pip install pytube

In [8]:
from pytube import YouTube
from moviepy import VideoFileClip

In [9]:
def download_youtube_video(url, download_path='downloads_vdo/'):
    # Create directory if it doesn't exist
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Download the YouTube video
    yt = YouTube(url)
    video = yt.streams.filter(only_video=False, file_extension='mp4').first()
    video_path = video.download(output_path=download_path)

    print(f"Video downloaded: {video_path}")
    return video_path

In [10]:
video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
                "https://www.youtube.com/watch?v=_p58cZIHDG4"]

In [ ]:
#for video in video_urls:
    #download_youtube_video(video)

In [ ]:
video_path = "/home/pms/llm_project/mm_rag_esg_financial_project/downloads_vdo/Invest_with_Conscience_ ESG_Investing.mp4"
audio_path = os.path.splitext(video_path)[0] + '.mp3'

video_clip = VideoFileClip(video_path)
video_clip.audio.write_audiofile(audio_path,codec='mp3')
video_clip.close()


In [ ]:
# class YouTubeAudioDownloader:
#     def __init__(self, output_folder):
#         self.output_folder = os.path.abspath(output_folder)
#         self.audio_files_dict = {}

#     def get_safe_filename(self, filename):
#         safe_filename = re.sub(r'[^\w\-.]', '_', filename)
#         safe_filename = re.sub(r'_+', '_', safe_filename)
#         safe_filename = safe_filename[:50].strip('_')
#         return safe_filename

#     def download_audio(self, video_url):
#         try:
#             ydl_opts = {
#                 'format': 'bestaudio/best',
#                 'postprocessors': [{
#                     'key': 'FFmpegExtractAudio',
#                     'preferredcodec': 'mp3',
#                     'preferredquality': '192',
#                 }],
#                 'outtmpl': os.path.join(self.output_folder, '%(title)s.%(ext)s'),
#             }

#             with YoutubeDL(ydl_opts) as ydl:
#                 info = ydl.extract_info(video_url, download=True)
#                 filename = ydl.prepare_filename(info)
#                 base, ext = os.path.splitext(filename)
#                 new_file = base + '.mp3'

#             print(f"Audio file downloaded: {new_file}")
#             self.audio_files_dict[video_url] = new_file
#             return new_file
#         except Exception as e:
#             print(f"Error downloading audio from {video_url}: {str(e)}")
#             return None

#     def download_multiple_audios(self, video_urls):
#         for url in video_urls:
#             print(f"Processing video: {url}")
#             audio_file = self.download_audio(url)
#             if audio_file is None:
#                 print(f"Failed to download audio from video: {url}")
#         return self.audio_files_dict

In [ ]:
# downloader = YouTubeAudioDownloader(output_folder="./data")
# video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
#                 "https://www.youtube.com/watch?v=_p58cZIHDG4"]

# # Download audios from multiple videos
# audio_files = downloader.download_multiple_audios(video_urls)

# print("Downloaded audio files:")
# for audio_file in audio_files:
#     print(audio_file)

### Transcriptions Generation with Whisper

In [ ]:
!pip -qqq install openai-whisper

In [2]:
import whisper
import torch

In [ ]:
# class AudioTranscriber:
#     def __init__(self, input_folder):
#         self.input_folder = os.path.abspath(os.path.join(os.getcwd(), input_folder))
#         self.whisper_model = None
#         self.transcriptions_dict = {}

#     def transcribe_audio(self, audio_file):
#         try:
#             if not os.path.exists(audio_file):
#                 print(f"Audio file not found: {audio_file}")
#                 return None

#             file_size = os.path.getsize(audio_file)
#             if file_size == 0:
#                 print(f"Audio file is empty: {audio_file}")
#                 return None

#             transcription = self.whisper_model.transcribe(audio_file)
#             return transcription["text"]
#         except Exception as e:
#             print(f"Error in transcribe_audio: {str(e)}")
#             return None

#     def transcribe_all_audios(self, audio_files_dict):
#         # for url, audio_path in audio_files_dict.items():
#         #     if not audio_path.endswith('.mp3'):
#         #         print(f"Skipping non-mp3 file: {audio_path}")
#         #         continue

#         transcription = self.transcribe_audio(audio_path)

#         if transcription is not None:
#             # Add to transcriptions dictionary
#             self.transcriptions_dict = {
#                 #'url': url,
#                 'audio_path': audio_path,
#                 'transcription': transcription
#             }
#         else:
#             print(f"Failed to transcribe audio: {audio_path}")

#         return self.transcriptions_dict

In [12]:
device = "cude" if torch.cuda.is_available() else "cpu"
whisper_model = whisper.load_model("medium",device=device)

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch

In [13]:
import glob
# Initialize the AudioTranscriber
#transcriber = AudioTranscriber(input_folder=r"./data")

# Initialize your Whisper model
#transcriber.whisper_model = whisper_model

# Transcribe all audios in the input folder
#for file in glob.glob(f'{os.getcwd()}/data/*.mp3'):
transcriptions_dict = whisper_model.transcribe("/home/pms/llm_project/mm_rag_esg_financial_project/data/ESG_investing_complete.mp3")

#print(transcriptions_dict)
#transcriptions_dict

# for _, data in transcriptions_dict.items():
#     #print(f"URL: {url}")
#     print(f"Audio file: {data['audio_path']}")
#     print(f"Transcription: {data['transcription'][:100]}...")  # Print first 100 characters
#     print("---")

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [14]:
transcriptions_dict['text']

" ESG, real or marketing? It's a complete fraud. Complete fraud. It's so ridiculous. Governance has been addressed. That's useful. But this idea that you're going to get a stamp that says, oh, listen, my supplier, I've offset their carbon credits and now I understand my... It's a joke. It's jargon. And I think what people are doing right now is using it as a way to, for example, if you can paint yourself as ESG, in Europe you can essentially borrow money from the ECB at negative rates. I can do you a massage now. It's a carry-trade. He doesn't want you touching him. Coronavirus. Go Chamath. Go Chamath. Hold on. But I personally believe in climate change. I know. I think we need to do something. And so the problem with ESG is it's going to take years for this... But when you hear JP Morgan yesterday say they're not going to finance fossil fuels or you hear Ed Bastien at Delta say he's going to spend $100 million, real money by the way, effectively buying carbon offsets and investing in 

### PDF Dpcument Parsing

In [1]:
from unstructured.partition.pdf import partition_pdf

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
esg_report_path = "/home/pms/llm_project/mm_rag_esg_financial_project/pdf/Bidding_Document_UWS_AGRC.pdf"

In [3]:
raw_pdf_elements=partition_pdf(
    filename="/home/pms/llm_project/mm_rag_esg_financial_project/pdf/Bidding_Document_UWS_AGRC.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
)

In [ ]:
raw_pdf_elements

In [ ]:
# Header=[]
# Footer=[]
# Title=[]
# NarrativeText=[]
# Text=[]
# ListItem=[]
# img=[]
# tab=[]
# for element in raw_pdf_elements:
#   if "unstructured.documents.elements.Header" in str(type(element)):
#             Header.append(str(element))
#   elif "unstructured.documents.elements.Footer" in str(type(element)):
#             Footer.append(str(element))
#   elif "unstructured.documents.elements.Title" in str(type(element)):
#             Title.append(str(element))
#   elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
#             NarrativeText.append(str(element))
#   elif "unstructured.documents.elements.Text" in str(type(element)):
#             Text.append(str(element))
#   elif "unstructured.documents.elements.ListItem" in str(type(element)):
#             ListItem.append(str(element))
#   elif "unstructured.documents.elements.Image" in str(type(element)):
#             img.append(str(element))
#   elif "unstructured.documents.elements.Table" in str(type(element)):
#             tab.append(str(element))

### Extracting Textual Component

In [16]:
from unstructured.documents.elements import NarrativeText

In [20]:
def extract_text_with_metadata(raw_data, source_document):

    text_data = []
    paragraph_counters = {}

    for element in raw_data:
        if isinstance(element, NarrativeText):
            page_number = element.metadata.page_number

            if page_number not in paragraph_counters:
                paragraph_counters[page_number] = 1
            else:
                paragraph_counters[page_number] += 1

            paragraph_number = paragraph_counters[page_number]

            text_content = element.text
            text_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "paragraph_number": paragraph_number,
                "text": text_content
            })

    return text_data

In [21]:
extracted_text = extract_text_with_metadata(raw_pdf_elements, esg_report_path)

In [ ]:
extracted_text

### Extracting Image Components

In [23]:
from unstructured.documents.elements import Image

In [24]:
def extract_image_metadata(raw_data, source_document):
    image_data = []

    for element in raw_data:
        if isinstance(element, Image):
            page_number = element.metadata.page_number
            image_path = element.metadata.image_path if hasattr(element.metadata, 'image_path') else None

            image_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "image_path": image_path
            })

    return image_data

In [25]:
extracted_image_data = extract_image_metadata(raw_pdf_elements, esg_report_path)

In [26]:
import matplotlib.pyplot as plt
from PIL import Image
import math

In [27]:
def display_images_from_metadata(extracted_image_data, images_per_row=4):
    valid_images = [img for img in extracted_image_data if img['image_path']]
    if not valid_images:
        print("No valid image data available.")
        return

    num_images = len(valid_images)
    num_rows = math.ceil(num_images / images_per_row)

    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(20, 5*num_rows))
    axes = axes.flatten() if num_rows > 1 else [axes]

    for ax, img_data in zip(axes, valid_images):
        try:
            img = Image.open(img_data['image_path'])
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(f"Page {img_data['page_number']}", fontsize=10)
        except Exception as e:
            print(f"Error loading image {img_data['image_path']}: {str(e)}")
            ax.text(0.5, 0.5, f"Error loading image\n{str(e)}", ha='center', va='center')
            ax.axis('off')

    for ax in axes[num_images:]:
        fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

In [ ]:
display_images_from_metadata(extracted_image_data)

### Extracting Table Components

In [29]:
from unstructured.documents.elements import Table

In [30]:
def extract_table_metadata(raw_data, source_document):
    table_data = []

    for element in raw_data:
        if isinstance(element, Table):
            page_number = element.metadata.page_number

            # Extract table content as a string
            table_content = str(element)

            table_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "table_content": table_content
            })

    return table_data

In [31]:
extracted_table_data = extract_table_metadata(raw_pdf_elements, esg_report_path)

In [8]:
import os
import json
config_data = json.load(open("./config.json"))
#MISTRALAI_API_KEY = config_data["MISTRAL_API_KEY"]
GROQ_API_KEY = config_data["GROQ_API_KEY"]
#os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llava-v1.5-7b-4096-preview",
    temperature=0.5,
    api_key=GROQ_API_KEY,
    max_tokens=4096,
    max_retries=2
)